## This notebook does the followings:
### + read sentence csv file, tokenize the assertions 
### + create list of tokens, word lenght and word frequency lists (based on wordnet) exported as orig_df.to_csv('processed_data/processed_sentences_tokenize_cloze_etc.csv')
### + also calculates cloze probabilities, entropy etc.  based on GPT-2  exported to processed_data/gpt_cloze_probabilities.csv', 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import ast

/tmp/ipykernel_837306/2145724294.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#orig_sentence_file = "exported_data/sentences_All-in-one_01Feb23.xlsx"
orig_sentence_file = "exported_data/sentences_23Jan24.xlsx"

orig_df = pd.read_excel(open(orig_sentence_file, 'rb'), sheet_name='stimuli') 

In [3]:
orig_df.head()

,Sno,Condition,Cond_remark,ID,orig_ID,Ling_type,Comp,remarks_2,remarks_1,processed_assertion,mw_mark,H_P_mw,H_N_mw,change_remark,image_name,aoi_name,overlay_name,Check
0,1,1,hate_w_high,27,140,lexical,1.0,comp,HighHateLowDiff,Frauen haben einen signifikant geringeren Durc...,0.0,"[5,6]",[1],no_change,img_1.png,img_1.ias,img_overlay_1.bmp,1
1,2,1,hate_w_high,26,196,lexical,0.0,derog,HighHateLowDiff,Frauen sind psychisch nicht für komplexe Arbei...,1.0,[4],[1],no_change,img_2.png,img_2.ias,img_overlay_2.bmp,1
2,3,1,hate_w_high,16,144,lexical,1.0,comp,HighHateLowDiff,Frauen haben im statistischen Mittel ein klein...,1.0,"[7,8]",[1],no_change,img_3.png,img_3.ias,img_overlay_3.bmp,1
3,4,1,hate_w_high,14,233,lexical,0.0,derog,HighHateLowDiff,Frauenförderung hat fast nur Nachteile für die...,0.0,[5],[1],no_change,img_4.png,img_4.ias,img_overlay_4.bmp,1
4,5,1,hate_w_high,2,175,lexical,1.0,comp,HighHateLowDiff,Frauen sind im Durchschnitt dümmer als Männer.,0.0,[5],"[1,7]",no_change,img_5.png,img_5.ias,img_overlay_5.bmp,1


In [4]:
orig_df.tail(50)

,Sno,Condition,Cond_remark,ID,orig_ID,Ling_type,Comp,remarks_2,remarks_1,processed_assertion,mw_mark,H_P_mw,H_N_mw,change_remark,image_name,aoi_name,overlay_name,Check
44,46,6,hate_m,502,502,lexical,0.0,other,generated,Männer sind nicht die hellsten.,2.0,[99],[99],generated,img_46.png,img_46.ias,img_overlay_46.bmp,1
45,47,6,hate_m,37,37,social,0.0,other,HighHateLowDiff,Der Mann verprasst das Geld.,2.0,[99],[99],generated,img_47.png,img_47.ias,img_overlay_47.bmp,1
46,48,6,hate_m,1,503,social,0.0,other,generated,Männer gehören auf den Bau.,NaN,"[1, 5]","[1,5]",generated,img_48.png,img_48.ias,img_overlay_48.bmp,1
47,49,6,hate_m,504,504,social,0.0,other,generated,Männer sind bei Stellenvergaben privilegiert.,2.0,[99],[99],generated,img_49.png,img_49.ias,img_overlay_49.bmp,1
48,50,6,hate_m,505,505,social,0.0,other,generated,Männer verbingen wenig Zeit mit ihren Kindern.,2.0,[99],[99],generated,img_50.png,img_50.ias,img_overlay_50.bmp,1
49,51,6,hate_m,506,506,social,0.0,derog,generated,Alle Männer sind ausgewachsene Kinder.,2.0,[99],[99],generated,img_51.png,img_51.ias,img_overlay_51.bmp,1
50,52,2,neutral_w,27,140,lexical,1.0,comp,HighHateLowDiff-Neutral,Pokemons haben einen signifikant geringeren Du...,0.0,[99],[1],subject,img_52.png,img_52.ias,img_overlay_52.bmp,1
51,53,2,neutral_w,26,196,lexical,0.0,dep,HighHateLowDiff-Neutral,Kinder sind psychisch nicht für komplexe Arbei...,1.0,[99],[1],subject,img_53.png,img_53.ias,img_overlay_53.bmp,1
52,54,2,neutral_w,16,144,lexical,1.0,comp,HighHateLowDiff-Neutral,Kinder haben im statistischen Mittel ein klein...,1.0,[99],[1],subject,img_54.png,img_54.ias,img_overlay_54.bmp,1
53,55,2,neutral_w,14,233,lexical,0.0,derog,HighHateLowDiff-Neutral,Egoismus hat fast nur Nachteile für die Gesell...,0.0,[99],[1],Subject,img_55.png,img_55.ias,img_overlay_55.bmp,1


In [5]:
#orig_df.drop(orig_df.tail(1).index,inplace=True)

-------------------------------------------------

### SENTENCE/TOKEN PREPROCESSING

In [6]:
import nltk
from nltk.tokenize import word_tokenize

In [7]:
freq_df = pd.read_csv('processed_data/dewiki-wordrank_result.txt', delimiter='\t', header=None ) # columns=['word', 'freq']
freq_df.columns = ['word', 'freq']


In [8]:
freq_df.loc[freq_df['word']== 'der','freq'].values[0]

34671177

In [9]:
def remove_items_from_list(orig_list, items_to_remove):
    return [i for i in orig_list if not i in items_to_remove]

In [10]:
#orig_df.loc[orig_df['processed_assertion']=='Nachteulen sind für bestimmte Berufe nich geeignet.', 'assertion']= 'Nachteulen sind für bestimmte Berufe nicht geeignet.'
orig_df.loc[orig_df['processed_assertion']=='Nachteulen sind für bestimmte Berufe nich geeignet.']

,Sno,Condition,Cond_remark,ID,orig_ID,Ling_type,Comp,remarks_2,remarks_1,processed_assertion,mw_mark,H_P_mw,H_N_mw,change_remark,image_name,aoi_name,overlay_name,Check


In [11]:
orig_df['tokens'] = np.nan
orig_df['tokens'] = orig_df['tokens'].astype('object')

orig_df['cleaned_tokens'] = np.nan
orig_df['cleaned_tokens'] = orig_df['cleaned_tokens'].astype('object')

orig_df['word_length'] = np.nan
orig_df['word_length'] = orig_df['word_length'].astype('object')

orig_df['word_freq'] = np.nan
orig_df['word_freq'] = orig_df['word_freq'].astype('object')

items_to_remove = ['.', ',']
for index, row in orig_df.iterrows():
    try:
        sentence = row['processed_assertion']
        
        tokens = nltk.word_tokenize(sentence.lower(), language='german')
        cleaned_tokens = remove_items_from_list(tokens, items_to_remove)
        #we don't want to have a special token for # e.g. in #metoo-bewegung
        if "#" in cleaned_tokens:
            
            sp_index = cleaned_tokens.index('#')
            #
            #print(sp_index, ''.join(['#', cleaned_tokens[sp_index+1]]))
            cleaned_tokens[sp_index+1] = ''.join(['#', cleaned_tokens[sp_index+1]])
            cleaned_tokens.remove("#")
        
        #print(tokens)
        orig_df.at[index, 'tokens'] = tokens
        orig_df.at[index, 'cleaned_tokens'] = cleaned_tokens
        word_length_list = []
        word_freq_list =[]
        
        for token in tokens:
            #print(token)
            word_length_list.append(len(token))
            
            if len(freq_df.loc[freq_df['word']== token,'freq']) >0:
                word_freq = freq_df.loc[freq_df['word']== token,'freq'].values[0]
            else:
                word_freq = None     
            word_freq_list.append(word_freq)
            
        #print(word_freq_list, word_length_list) 
           
        orig_df.at[index, 'word_length'] = word_length_list
        orig_df.at[index, 'word_freq'] = word_freq_list
    except:
        print('Problem with the sentence :', index)    

In [12]:
orig_df.loc[orig_df['processed_assertion']=='Die #metoo-Bewegung ist hysterisch.']                     

,Sno,Condition,Cond_remark,ID,orig_ID,Ling_type,Comp,remarks_2,remarks_1,processed_assertion,...,H_N_mw,change_remark,image_name,aoi_name,overlay_name,Check,tokens,cleaned_tokens,word_length,word_freq
9,10,1,hate_w_high,31,44,lexical,0.0,derog,HighHateLowDiff,Die #metoo-Bewegung ist hysterisch.,...,[3],no_change,img_10.png,img_10.ias,img_overlay_10.bmp,1,"[die, #, metoo-bewegung, ist, hysterisch, .]","[die, #metoo-bewegung, ist, hysterisch]","[3, 1, 14, 3, 10, 1]","[26485493, None, None, 6149650, 272, None]"


In [13]:
orig_df.columns

Index(['Sno', 'Condition', 'Cond_remark', 'ID', 'orig_ID', 'Ling_type', 'Comp',
       'remarks_2', 'remarks_1', 'processed_assertion', 'mw_mark', 'H_P_mw',
       'H_N_mw', 'change_remark', 'image_name', 'aoi_name', 'overlay_name',
       'Check', 'tokens', 'cleaned_tokens', 'word_length', 'word_freq'],
      dtype='object')

In [14]:
orig_df.loc[9, 'cleaned_tokens']

['die', '#metoo-bewegung', 'ist', 'hysterisch']

In [15]:
orig_df.loc[59, 'cleaned_tokens']

['die', '#keinurlaub-bewegung', 'ist', 'hysterisch']

In [16]:
orig_df['cleaned_token_count'] = orig_df['cleaned_tokens'].apply(len)

In [17]:
orig_df.loc[5,'H_N_mw']

'[1,3]'

In [18]:
#ast.literal_eval(orig_df.loc[2,'H_N_mw'])

orig_df['H_N_mw']= orig_df['H_N_mw'].apply(lambda x: ast.literal_eval(x))
orig_df['H_P_mw']= orig_df['H_P_mw'].apply(lambda x: ast.literal_eval(x))

In [19]:
orig_df.loc[5,'H_N_mw']

[1, 3]

In [20]:
orig_df.columns

Index(['Sno', 'Condition', 'Cond_remark', 'ID', 'orig_ID', 'Ling_type', 'Comp',
       'remarks_2', 'remarks_1', 'processed_assertion', 'mw_mark', 'H_P_mw',
       'H_N_mw', 'change_remark', 'image_name', 'aoi_name', 'overlay_name',
       'Check', 'tokens', 'cleaned_tokens', 'word_length', 'word_freq',
       'cleaned_token_count'],
      dtype='object')

In [21]:
orig_df.to_csv('processed_data/processed_sentences_tokenize_cloze_etc.csv', sep='\t', index=False)


orig_df.to_pickle('processed_data/processed_sentences_tokenize_cloze_etc.pkl')  ## it is easier to read the cells with list values

-------------------------------------------------

# CLOZE PROBABILITIES FROM GPT-2

In [22]:
cloze_df= pd.DataFrame(columns=['Sno', 'Ino', 'tokens', 'sentence', 'masked_token'])
for index, row in orig_df.iterrows():
    sentence= row['processed_assertion'].replace('.', '')
    tokens = sentence.split(' ')
    inc_tokens = []
    for i, value in enumerate(tokens):
        print(i, value)
        if i < len(tokens)-1:
            inc_tokens.append(value)
            new_row= len(cloze_df)
            
            cloze_df.at[new_row, 'sentence']=" ".join(inc_tokens) #+ ' [MASK]'
            cloze_df.at[new_row, 'Sno']=row['Sno']
            cloze_df.at[new_row, 'Ino']=i
            cloze_df.at[new_row, 'tokens']=inc_tokens
            cloze_df.at[new_row, 'masked_token']=tokens[i+1]
        
cloze_df.to_csv("sentences_cloze.csv")  

0 Frauen
1 haben
2 einen
3 signifikant
4 geringeren
5 Durchsetzungswillen
0 Frauen
1 sind
2 psychisch
3 nicht
4 für
5 komplexe
6 Arbeit
7 geschaffen
0 Frauen
1 haben
2 im
3 statistischen
4 Mittel
5 ein
6 kleineres
7 Allgemeinwissen
0 Frauenförderung
1 hat
2 fast
3 nur
4 Nachteile
5 für
6 die
7 Gesellschaft
0 Frauen
1 sind
2 im
3 Durchschnitt
4 dümmer
5 als
6 Männer
0 Frauen
1 sind
2 Männern
3 geistig
4 in
5 einigen
6 Bereichen
7 unterlegen
0 Frauen
1 sind
2 Minderleister
3 in
4 unserer
5 Gesellschaft
0 Frauen
1 können
2 nichts
3 und
4 sind
5 zu
6 dumm
0 Frauen
1 ruinieren
2 oft
3 den
4 größten
5 Spaß
0 Die
1 #metoo-Bewegung
2 ist
3 hysterisch
0 Frauen
1 sollten
2 zu
3 Hause
4 bleiben
5 und
6 sich
7 um
8 das
9 Kind
10 kümmern
0 Feministen
1 sind
2 überwiegend
3 angetrieben
4 durch
5 Männerhass
0 Frauen
1 altern
2 wie
3 Milch,
4 Männer
5 altern
6 wie
7 Wein
0 Eine
1 klassische
2 Rollenverteilung
3 in
4 der
5 Ehe
6 ist
7 zu
8 bevorzugen
0 Frauen
1 sind
2 für
3 gewisse
4 Berufen
5 nicht
6 

In [23]:
cloze_df.head(20)  

,Sno,Ino,tokens,sentence,masked_token
0,1,0,"[Frauen, haben, einen, signifikant, geringeren]",Frauen,haben
1,1,1,"[Frauen, haben, einen, signifikant, geringeren]",Frauen haben,einen
2,1,2,"[Frauen, haben, einen, signifikant, geringeren]",Frauen haben einen,signifikant
3,1,3,"[Frauen, haben, einen, signifikant, geringeren]",Frauen haben einen signifikant,geringeren
4,1,4,"[Frauen, haben, einen, signifikant, geringeren]",Frauen haben einen signifikant geringeren,Durchsetzungswillen
5,2,0,"[Frauen, sind, psychisch, nicht, für, komplexe...",Frauen,sind
6,2,1,"[Frauen, sind, psychisch, nicht, für, komplexe...",Frauen sind,psychisch
7,2,2,"[Frauen, sind, psychisch, nicht, für, komplexe...",Frauen sind psychisch,nicht
8,2,3,"[Frauen, sind, psychisch, nicht, für, komplexe...",Frauen sind psychisch nicht,für
9,2,4,"[Frauen, sind, psychisch, nicht, für, komplexe...",Frauen sind psychisch nicht für,komplexe


In [24]:
cloze_df.tail(20) 

,Sno,Ino,tokens,sentence,masked_token
570,96,3,"[Ab, und, zu, eine, erzieherische, Ohrfeige, h...",Ab und zu eine,erzieherische
571,96,4,"[Ab, und, zu, eine, erzieherische, Ohrfeige, h...",Ab und zu eine erzieherische,Ohrfeige
572,96,5,"[Ab, und, zu, eine, erzieherische, Ohrfeige, h...",Ab und zu eine erzieherische Ohrfeige,hat
573,96,6,"[Ab, und, zu, eine, erzieherische, Ohrfeige, h...",Ab und zu eine erzieherische Ohrfeige hat,noch
574,96,7,"[Ab, und, zu, eine, erzieherische, Ohrfeige, h...",Ab und zu eine erzieherische Ohrfeige hat noch,niemand
575,96,8,"[Ab, und, zu, eine, erzieherische, Ohrfeige, h...",Ab und zu eine erzieherische Ohrfeige hat noch...,"geschadet,"
576,96,9,"[Ab, und, zu, eine, erzieherische, Ohrfeige, h...",Ab und zu eine erzieherische Ohrfeige hat noch...,auch
577,96,10,"[Ab, und, zu, eine, erzieherische, Ohrfeige, h...",Ab und zu eine erzieherische Ohrfeige hat noch...,nicht
578,96,11,"[Ab, und, zu, eine, erzieherische, Ohrfeige, h...",Ab und zu eine erzieherische Ohrfeige hat noch...,Mickey
579,96,12,"[Ab, und, zu, eine, erzieherische, Ohrfeige, h...",Ab und zu eine erzieherische Ohrfeige hat noch...,Mouse


In [25]:

# Using Series.values.tolist()
masked_sentence_list = cloze_df["sentence"].values.tolist()
label_list= cloze_df["masked_token"].values.tolist()

In [26]:
masked_sentence_list

['Frauen',
 'Frauen haben',
 'Frauen haben einen',
 'Frauen haben einen signifikant',
 'Frauen haben einen signifikant geringeren',
 'Frauen',
 'Frauen sind',
 'Frauen sind psychisch',
 'Frauen sind psychisch nicht',
 'Frauen sind psychisch nicht für',
 'Frauen sind psychisch nicht für komplexe',
 'Frauen sind psychisch nicht für komplexe Arbeit',
 'Frauen',
 'Frauen haben',
 'Frauen haben im',
 'Frauen haben im statistischen',
 'Frauen haben im statistischen Mittel',
 'Frauen haben im statistischen Mittel ein',
 'Frauen haben im statistischen Mittel ein kleineres',
 'Frauenförderung',
 'Frauenförderung hat',
 'Frauenförderung hat fast',
 'Frauenförderung hat fast nur',
 'Frauenförderung hat fast nur Nachteile',
 'Frauenförderung hat fast nur Nachteile für',
 'Frauenförderung hat fast nur Nachteile für die',
 'Frauen',
 'Frauen sind',
 'Frauen sind im',
 'Frauen sind im Durchschnitt',
 'Frauen sind im Durchschnitt dümmer',
 'Frauen sind im Durchschnitt dümmer als',
 'Frauen',
 'Frauen 

In [27]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, AutoModel, AutoConfig, utils
from transformers import AutoTokenizer, AutoModelWithLMHead

import torch
import pickle

import logging
import matplotlib.pyplot as plt
from torch.nn import functional as F
from scipy.stats import entropy
from statistics import mean, median

/home/oalacam/projects/gaze4hate/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
# Using GPT, compute probability over its entire vocabulary, returning logits, ranks, entropy_lists and attention maps
def get_gpt_predictions(masked_sentence_list, label_list, vocabulary=None):
    """
    Reproduces what get_bert_predictions() does for the gpt model
    (i.e. compute probability over its entire vocabulary, returning logits, ranks, entropy_lists and attention maps).
    
    If vocabulary = None (default), then the computed values for the (sub-)tokens refer to the entire GPT vocabulary.
    
    If vocabulary != None, then the computed values for the tokens are calculated only for the vocabulary
    that appeared in the human produced data, i.e. the vocabulary that is passed as "vocabulary"
    (*** --> I'm not so sure about these values here !! This is still work in progress***)
    
    """
    if vocabulary is not None:
        ### add tokens from the vocabulary to the tokenizer and resize model embeddings
        tokenizer = AutoTokenizer.from_pretrained("dbmdz/german-gpt2",output_hidden_states=True, return_dict=True, output_attentions=True)
        model = AutoModelWithLMHead.from_pretrained("dbmdz/german-gpt2",output_hidden_states=True, return_dict=True, output_attentions=True)
        #print("Add vocabulary to the tokenizer...")
        tokenizer.add_tokens(vocabulary)
        model.resize_token_embeddings(len(tokenizer))
    else:
        tokenizer = AutoTokenizer.from_pretrained("dbmdz/german-gpt2",output_hidden_states=True, return_dict=True, output_attentions=True)
        model = AutoModelWithLMHead.from_pretrained("dbmdz/german-gpt2",output_hidden_states=True, return_dict=True, output_attentions=True)

    scores=[]
    ranks=[]
    entropy_list =[]
    target_probabilities =[]
    target_prob=[]
    attention_list = []
    
    input_text_list = []
    target_text_list = []

    tokenizer.pad_token_id = tokenizer.eos_token_id
    
    for i,sentence in enumerate(masked_sentence_list):
        #print(i, "sentence:", sentence)
        
        input = tokenizer(sentence, return_tensors='pt', padding='longest')
        #print("sentence tokenized (input):", input)
        input_text = tokenizer.convert_ids_to_tokens(input['input_ids'][0])
        #print("input_ids converted to tokens (input_text):", input_text)
        
        ### mask index = place of the MASK token in the tensor;
        ### here it is an "x" token instead of MASK
        mask_index = len(input["input_ids"][0])-1  #named liked that so it mirrors  Özge's code
        #print("Mask index:", mask_index)
       
        ### encode the corresponding label from the label list as the target token
        #print("Label/Target token from the label list:", label_list[i])
        target_token = tokenizer(label_list[i], return_tensors = "pt")
        #print('Target token encoded: ', target_token)
        target_text = tokenizer.convert_ids_to_tokens(target_token['input_ids'][0])
        #print('Target text converted to tokens: ', target_text) 
        
        input_text_list.append(input_text)
        target_text_list.append(target_text)
        #print("input_text_list", input_text_list)
        #print("target_text_list", target_text_list)

        with torch.no_grad():
            output = model(**input, 
                          output_attentions=True)
        
        ### softmax/ get probabilities 
        softmax = F.softmax(output.logits[0], dim=-1)
        #print("softmax:", softmax)
        #print("dimension softmax:", softmax.size())
        maskP=[]
        
        ### attentions
        attention_scores = output.attentions#[0].detach().cpu().numpy() # extract the attention scores from all layers
        attention_list.append(attention_scores)
        #print("Attention scores:", attention_scores)
        
        ## initialize variables rank_min and avg_token_entropy
        rank_min = softmax.size(1) 
        avg_token_entropy = 0.0   
    
        ## now iterate over each token in the target_text
        for token in target_text:
            ## For each token, calculate the probability (tokenP)
            print('--------------------------') 
            print("Target token:", token)
            token_idx = tokenizer.encode(token)[0]
            print("token_idx:", token_idx)

            tokenP = softmax[mask_index, token_idx].item()
            #print("Token probability (for x-token position):", tokenP)
            ## i.e. probability of word to be the last token
            ## I replaced the MASK-token with an "x"
            
            ### Either: restrict calculation to given vocabulary (** not so sure about this option **)
            if vocabulary is not None:
                ## initialize mask_over_vocab_list as an array of zeros with the length of the vocabulary
                ## this array will store the probabilities of each token in the vocabulary at the "mask" position:
                mask_over_vocab_list = np.zeros(len(vocabulary))
                ## iterate over the token in the vocabulary:
                for j, vocab_token in enumerate(vocabulary):
                    # print(j, vocab_token)
                    ## obtain the token index of the current vocabulary token: 
                    vocab_token_idx = tokenizer.encode(vocab_token)[0]
                    ## use this index to access the corresponding probability in the softmax tensor at the x-position
                    ## and assign the probability of the vocabulary token to the corresponding index: 
                    mask_over_vocab_list[j] = softmax[mask_index, vocab_token_idx].item()
                    
            ### Or: calculate values for GPT's entire vocabulary...       
            else: 
                mask_over_vocab_list = softmax[torch.tensor([mask_index], dtype=torch.long)].numpy()[0]

            #### 
            #print("probabilities over the vocabulary for the mask position (mask over vocab list):", mask_over_vocab_list)
            #print("number of probabilities:", len(mask_over_vocab_list))
    
            token_entropy = entropy(mask_over_vocab_list)
            #print("entropy of these probabilities/ token entropy:", token_entropy)
            
            ## Update the average token entropy accordingly:
            avg_token_entropy += token_entropy
        
            ## sort the mask_over_vocab_list in descending order 
            sorted_array = np.sort(mask_over_vocab_list)
            reverse_array = list(sorted_array[::-1])    
                
            ## find the rank of the current token's probability (tokenP) in this sorted list. 
            ## Özge: "It is not the perfect solution , but it will give us a good estimate"
            rank = reverse_array.index(tokenP)
            #print("current token's rank:", rank)
            
            if rank <= rank_min:
                ## store the minimum rank encountered in the variable rank_min:
                rank_min = rank
                
            #print('Token and its rank: ', token, rank)
                
            ## append the tokenP to the maskP list.
            maskP.append(tokenP) 

        #### For each sentence in the sentence list:
        ## After iterating over all tokens, the average token entropy is divided by the number of tokens
        ## to compute the average token entropy: 
        if avg_token_entropy>0:
            
            avg_token_entropy = avg_token_entropy/(len(target_text)) 
        
        ## rank_min represents the rank of the target token with the lowest rank among all tokens:
        target_rank = rank_min
        
        ## the mean of the probabilities in maskP represents the target probability:
        print('maskP', maskP)
        if maskP != []:
            target_probability = mean(maskP)
        
        else:
            print('WARNING: maskp is empty list', tokenP)
            target_probability = 'warning'
        
        print(target_text)
        print('-- Target Probability (mean of probabilities in maskP): ', target_probability)
        print('-- Target Rank: ', target_rank)
        print('-- Target Entropy: ', avg_token_entropy)                   
        
        scores.append(target_probability)
        ranks.append(target_rank)
        entropy_list.append(avg_token_entropy)
        
        #print("\n \n \n")
            
    return scores, ranks, entropy_list, attention_list, input_text_list

In [29]:
scores, ranks, entropy_list, attention_list, input_text_list = get_gpt_predictions(masked_sentence_list, label_list, vocabulary=None)
print(scores)

/home/oalacam/projects/gaze4hate/venv/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1581: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


--------------------------
Target token: haben
token_idx: 4199
maskP [7.831689799786545e-06]
['haben']
-- Target Probability (mean of probabilities in maskP):  7.831689799786545e-06
-- Target Rank:  5047
-- Target Entropy:  5.7409138679504395
--------------------------
Target token: einen
token_idx: 946
maskP [5.625740868708817e-06]
['einen']
-- Target Probability (mean of probabilities in maskP):  5.625740868708817e-06
-- Target Rank:  5263
-- Target Entropy:  5.9237775802612305
--------------------------
Target token: sign
token_idx: 29597
--------------------------
Target token: if
token_idx: 1038
--------------------------
Target token: ikant
token_idx: 31257
maskP [4.024438737815217e-08, 6.941044716768374e-08, 3.6256195962147686e-11]
['sign', 'if', 'ikant']
-- Target Probability (mean of probabilities in maskP):  3.6563696913932686e-08
-- Target Rank:  25273
-- Target Entropy:  6.984785556793213
--------------------------
Target token: ger
token_idx: 452
--------------------------

In [30]:
cloze_df['probability'] = scores
cloze_df['ranks'] = ranks
cloze_df['entropy'] = entropy_list
#cloze_df['attention'] = attention_list
cloze_df['inputtext'] = input_text_list

In [31]:
cloze_df.to_csv('processed_data/gpt_cloze_probabilities.csv', encoding='utf-8')

In [29]:
cloze_df.tail(10)

,Sno,Ino,tokens,sentence,masked_token,probability,ranks,entropy,inputtext
580,97,0,"[Der, aktuelle, Drang, zur, Emanzipation, schl...",Der,aktuelle,0.000007,8331,8.252378,[Der]
581,97,1,"[Der, aktuelle, Drang, zur, Emanzipation, schl...",Der aktuelle,Drang,0.0,22247,7.080990,"[Der, Ġaktuelle]"
582,97,2,"[Der, aktuelle, Drang, zur, Emanzipation, schl...",Der aktuelle Drang,zur,0.000008,1594,2.960689,"[Der, Ġaktuelle, ĠDr, ang]"
583,97,3,"[Der, aktuelle, Drang, zur, Emanzipation, schl...",Der aktuelle Drang zur,Emanzipation,0.0,15611,7.016135,"[Der, Ġaktuelle, ĠDr, ang, Ġzur]"
584,97,4,"[Der, aktuelle, Drang, zur, Emanzipation, schl...",Der aktuelle Drang zur Emanzipation,schlägt,0.0,10851,4.642233,"[Der, Ġaktuelle, ĠDr, ang, Ġzur, ĠEm, anzip, a..."
585,97,5,"[Der, aktuelle, Drang, zur, Emanzipation, schl...",Der aktuelle Drang zur Emanzipation schlägt,teilweise,0.0,7930,3.521955,"[Der, Ġaktuelle, ĠDr, ang, Ġzur, ĠEm, anzip, a..."
586,97,6,"[Der, aktuelle, Drang, zur, Emanzipation, schl...",Der aktuelle Drang zur Emanzipation schlägt te...,um,0.000011,1482,3.971430,"[Der, Ġaktuelle, ĠDr, ang, Ġzur, ĠEm, anzip, a..."
587,97,7,"[Der, aktuelle, Drang, zur, Emanzipation, schl...",Der aktuelle Drang zur Emanzipation schlägt te...,in,0.000003,539,1.767482,"[Der, Ġaktuelle, ĠDr, ang, Ġzur, ĠEm, anzip, a..."
588,97,8,"[Der, aktuelle, Drang, zur, Emanzipation, schl...",Der aktuelle Drang zur Emanzipation schlägt te...,eine,0.000008,4836,4.868682,"[Der, Ġaktuelle, ĠDr, ang, Ġzur, ĠEm, anzip, a..."
589,97,9,"[Der, aktuelle, Drang, zur, Emanzipation, schl...",Der aktuelle Drang zur Emanzipation schlägt te...,Rachsucht,0.000001,10705,7.187602,"[Der, Ġaktuelle, ĠDr, ang, Ġzur, ĠEm, anzip, a..."


In [ ]:
### add manipulated words info as columns
grouped_df = cloze_df.groupby(['Sno' 'Ino'], dropna=False)


for group_name in grouped_df.groups.keys(): # sample key : ('p1', 5)
    print('Processing : ',group_name)
    trial_df = grouped_df.get_group(group_name)
    sen_score = trial_df.entropy.mean()